In [ ]:
# idea: enumerate all relevant pairings, construct the linear equation and get the points which are a certain distance away

In [120]:
from itertools import combinations
import numpy as np
import copy

In [121]:
test = False
if test:
    text = """............
........0...
.....0......
.......0....
....0.......
......A.....
............
............
........A...
.........A..
............
............"""
else:
    with open('08_input.txt', 'r') as file:
        text = file.read()

antenna_map = [list(x) for x in text.split("\n")]

In [122]:
# for storage, I want a dict where each char is a key and each value is a list of coordinates where this char occurs
location_dict = dict()
for y in range(len(antenna_map)):
    for x in range(len(antenna_map[y])):
        char = antenna_map[y][x]
        if char != '.':
            if char not in location_dict.keys():
                location_dict[char] = [np.array([x, y])]
            else:
                location_dict[char].append(np.array([x, y]))
location_dict

{'0': [array([8, 0]), array([7, 2]), array([18,  7]), array([9, 9])],
 'c': [array([32,  0]), array([41,  4]), array([29,  5])],
 'a': [array([8, 1]), array([7, 5]), array([3, 8]), array([ 0, 28])],
 'r': [array([48,  2]), array([38,  5]), array([37,  7])],
 'W': [array([5, 3]), array([11, 10]), array([ 6, 11]), array([ 0, 12])],
 'Z': [array([14,  4]), array([47,  8]), array([ 9, 13]), array([24, 15])],
 'F': [array([17,  4]), array([2, 5]), array([ 6, 22])],
 'L': [array([37,  5]), array([36,  6]), array([41,  8]), array([30, 13])],
 '5': [array([43,  5]), array([37,  6]), array([36,  8]), array([20, 21])],
 'v': [array([28,  6]), array([24,  7]), array([35, 17]), array([33, 32])],
 'E': [array([41,  7]), array([33,  8]), array([38,  9])],
 'p': [array([30,  8]), array([28,  9]), array([27, 12]), array([19, 17])],
 '7': [array([40,  8]), array([13, 10]), array([28, 21]), array([29, 27])],
 'm': [array([43,  8]), array([31, 15]), array([36, 17]), array([42, 30])],
 'j': [array([6, 9])

# Task 1

In [123]:
# now, we can go through each pairwise combination and get the corresponding antinode positions
antinodes = []
map_bounds = 0, len(antenna_map[0]), 0, len(antenna_map) # xmin, xmax, ymin, ymax

for char in location_dict.keys():
    locations = location_dict[char]
    location_combinations = list(combinations(locations, 2))
    for points in location_combinations:
        p1, p2 = points[0], points[1]
        delta = p2 - p1  # dx, dy
        # adding/subtracting the deltas to p1 and p2
        # then getting rid of p1 and p2
        inferred_p1 = p1 + delta
        inferred_p2 = p1 - delta
        inferred_p3 = p2 + delta
        inferred_p4 = p2 - delta

        for inferred_point in [inferred_p1, inferred_p2, inferred_p3, inferred_p4]:
            # checking if the inferred point is one of the original points
            if np.any(inferred_point != p1) and np.any(inferred_point != p2):
                # checking if the inferred point is in bounds
                if inferred_point[0] >= map_bounds[0] and inferred_point[0] < map_bounds[1] and inferred_point[1] >= map_bounds[2] and inferred_point[1] < map_bounds[3]:
                    antinodes.append(inferred_point)

# removing duplicates
antinodes = np.unique(np.vstack(antinodes), axis=0)
len(antinodes)

409

# Task 2

In [124]:
# now, we can go through each pairwise combination and get the corresponding antinode positions
# for storage, I want a dict where each char is a key and each value is a list of coordinates where this char occurs
location_dict = dict()
for y in range(len(antenna_map)):
    for x in range(len(antenna_map[y])):
        char = antenna_map[y][x]
        if char != '.':
            if char not in location_dict.keys():
                location_dict[char] = [np.array([x, y])]
            else:
                location_dict[char].append(np.array([x, y]))

antinodes = []
map_bounds = 0, len(antenna_map[0]), 0, len(antenna_map) # xmin, xmax, ymin, ymax

for char in location_dict.keys():
    locations = location_dict[char]
    location_combinations = list(combinations(locations, 2))
    
    for points in location_combinations:
        p1, p2 = points[0], points[1]
        delta = p2 - p1  # dx, dy
                
        # need to do this until we go out of bounds, but can only do from one points perspective
        # getting the negative antinodes
        in_bounds = True
        inferred_point = p1.copy()
        # p1 and p2 are in the points automatically
        antinodes.append(p1.copy())
        while in_bounds:
            inferred_point -= delta
            
            if inferred_point[0] >= map_bounds[0] and inferred_point[0] < map_bounds[1] and inferred_point[1] >= map_bounds[2] and inferred_point[1] < map_bounds[3]:
                antinodes.append(inferred_point.copy())
                    
            else:
                in_bounds = False
                
        in_bounds = True
        inferred_point = p1.copy()
        while in_bounds:
            inferred_point += delta
            
            if inferred_point[0] >= map_bounds[0] and inferred_point[0] < map_bounds[1] and inferred_point[1] >= map_bounds[2] and inferred_point[1] < map_bounds[3]:
                antinodes.append(inferred_point.copy())
            else:
                in_bounds = False
                
# removing duplicates
antinodes = np.unique(np.vstack(antinodes), axis=0)
len(antinodes)

1308

In [101]:
new_map = antenna_map.copy()
for x, y in antinodes:
    new_map[y][x] = '#'

[['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.', '.', '0', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '0', '.', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.', '0', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '0', '.', '.', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', 'A', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.', '.', 'A', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.', '.', '.', 'A', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']]

In [107]:
# just for debug purposes
def print_map():
    updated_map = copy.deepcopy(antenna_map)
    for x, y in antinodes:
        updated_map[y][x] = '#'
    return '\n'.join([''.join(x) for x in updated_map])
computed_map = print_map()
computed_map

'##....#....#\n.#.#....#...\n..#.##....#.\n..##...#....\n....#....#..\n.#...##....#\n...#..#.....\n#....#.#....\n..#.....#...\n....#....#..\n.#........#.\n...#......##'

In [106]:
solution = """##....#....#
.#.#....0...
..#.#0....#.
..##...0....
....0....#..
.#...#A....#
...#..#.....
#....#.#....
..#.....A...
....#....A..
.#........#.
...#......##"""

solution

'##....#....#\n.#.#....0...\n..#.#0....#.\n..##...0....\n....0....#..\n.#...#A....#\n...#..#.....\n#....#.#....\n..#.....A...\n....#....A..\n.#........#.\n...#......##'

In [109]:
for char in range(len(solution)):
    if computed_map[char] != solution[char]:
        print(f"Mismatch at {char}: {computed_map[char]} - {solution[char]}")

Mismatch at 21: # - 0
Mismatch at 31: # - 0
Mismatch at 46: # - 0
Mismatch at 56: # - 0
Mismatch at 71: # - A
Mismatch at 112: # - A
Mismatch at 126: # - A


In [115]:
np.unique([list(x) for x in solution.split("\n")], return_counts=True)

(array(['#', '.', '0', 'A'], dtype='<U1'), array([ 27, 110,   4,   3]))

In [116]:
27+4+3

34